In [5]:
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime
import numpy as np
import ssl

# A data-mining exercise
# Let's see what happens when we just trade repeatedly off of technical
# trend indicators. On a given day, using data from the past X years, identify
# the top 5 historical trades in a given index, based off of some criteria such
# as Sharpe.

# Pre-process data with forward filled fields, in this case using the SP500
# Read in the SP500 tickers from S&P500-Symbols.csv, which was pulled from
# Wikipedia and saved locally.

# Run the below code to pull symbols
# ssl._create_default_https_context = ssl._create_unverified_context
# table=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
# df = table[0]
# df.to_csv("S&P500-Symbols.csv", columns=['Symbol'], index=False)

# Pull the adjusted close prices off Yahoo Finance
# df = pd.read_csv("S&P500-Symbols.csv")
# tickers = list(df['Symbol'])
start_date = '1989-01-01'
end_date = '2024-01-03'  # Get data a few days past end of year to backfill

# Pull off Yahoo finance the adjusted close prices
# data = pd.DataFrame(yf.download(tickers, start_date, end_date)['Adj Close'])
data.to_csv("S&P500-adjusted-close.csv")
data = 
all_dates = pd.date_range(start_date, end_date)


In [11]:
data = pd.DataFrame(yf.download(tickers, start_date, end_date)['Adj Close'])

all_dates = pd.date_range(start_date, end_date)
data['Price Date'] = data.index

# Fill in missing prices with the first price after the date
# This would be "forward-looking" if we were trading off of prices, but
# we are trading off of specific dates so we do not see into the future
data = data.reindex(all_dates, method='bfill')

# Drop data for 2024
data = data[spx.index < '2024-01-01']

[*********************100%%**********************]  503 of 503 completed

2 Failed downloads:
['BF.B']: Exception('%ticker%: No price data found, symbol may be delisted (1d 1989-01-01 -> 2024-01-03)')
['BRK.B']: Exception('%ticker%: No timezone found, symbol may be delisted')
/var/folders/p9/gj51q_sn5_v2x1kz79n76qzh0000gn/T/ipykernel_90654/2722538064.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data['Price Date'] = data.index


NameError: name 'spx' is not defined

In [12]:
data

,A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS,Price Date
1989-01-01,NaN,NaN,0.282736,NaN,NaN,1.224257,NaN,NaN,1.437745,1.320041,...,3.112027,3.285615,0.475442,NaN,NaN,NaN,NaN,1.350708,NaN,1989-01-03
1989-01-02,NaN,NaN,0.282736,NaN,NaN,1.224257,NaN,NaN,1.437745,1.320041,...,3.112027,3.285615,0.475442,NaN,NaN,NaN,NaN,1.350708,NaN,1989-01-03
1989-01-03,NaN,NaN,0.282736,NaN,NaN,1.224257,NaN,NaN,1.437745,1.320041,...,3.112027,3.285615,0.475442,NaN,NaN,NaN,NaN,1.350708,NaN,1989-01-03
1989-01-04,NaN,NaN,0.294115,NaN,NaN,1.243488,NaN,NaN,1.496428,1.346443,...,3.148075,3.342591,0.475442,NaN,NaN,NaN,NaN,1.350708,NaN,1989-01-04
1989-01-05,NaN,NaN,0.295866,NaN,NaN,1.240283,NaN,NaN,1.467087,1.333242,...,3.136056,3.371079,0.458462,NaN,NaN,NaN,NaN,1.305178,NaN,1989-01-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-30,138.75,13.44,185.639999,158.30748,134.479996,109.321869,75.550003,345.657806,580.070007,193.580002,...,63.470001,102.360001,35.630001,113.669998,129.029999,121.389999,267.980011,44.139999,196.125183,2024-01-02
2023-12-31,138.75,13.44,185.639999,158.30748,134.479996,109.321869,75.550003,345.657806,580.070007,193.580002,...,63.470001,102.360001,35.630001,113.669998,129.029999,121.389999,267.980011,44.139999,196.125183,2024-01-02
2024-01-01,138.75,13.44,185.639999,158.30748,134.479996,109.321869,75.550003,345.657806,580.070007,193.580002,...,63.470001,102.360001,35.630001,113.669998,129.029999,121.389999,267.980011,44.139999,196.125183,2024-01-02
2024-01-02,138.75,13.44,185.639999,158.30748,134.479996,109.321869,75.550003,345.657806,580.070007,193.580002,...,63.470001,102.360001,35.630001,113.669998,129.029999,121.389999,267.980011,44.139999,196.125183,2024-01-02


In [10]:
tickers

['MMM',
 'AOS',
 'ABT',
 'ABBV',
 'ACN',
 'ADBE',
 'AMD',
 'AES',
 'AFL',
 'A',
 'APD',
 'ABNB',
 'AKAM',
 'ALB',
 'ARE',
 'ALGN',
 'ALLE',
 'LNT',
 'ALL',
 'GOOGL',
 'GOOG',
 'MO',
 'AMZN',
 'AMCR',
 'AEE',
 'AAL',
 'AEP',
 'AXP',
 'AIG',
 'AMT',
 'AWK',
 'AMP',
 'AME',
 'AMGN',
 'APH',
 'ADI',
 'ANSS',
 'AON',
 'APA',
 'AAPL',
 'AMAT',
 'APTV',
 'ACGL',
 'ADM',
 'ANET',
 'AJG',
 'AIZ',
 'T',
 'ATO',
 'ADSK',
 'ADP',
 'AZO',
 'AVB',
 'AVY',
 'AXON',
 'BKR',
 'BALL',
 'BAC',
 'BK',
 'BBWI',
 'BAX',
 'BDX',
 'BRK.B',
 'BBY',
 'BIO',
 'TECH',
 'BIIB',
 'BLK',
 'BX',
 'BA',
 'BKNG',
 'BWA',
 'BXP',
 'BSX',
 'BMY',
 'AVGO',
 'BR',
 'BRO',
 'BF.B',
 'BLDR',
 'BG',
 'CDNS',
 'CZR',
 'CPT',
 'CPB',
 'COF',
 'CAH',
 'KMX',
 'CCL',
 'CARR',
 'CTLT',
 'CAT',
 'CBOE',
 'CBRE',
 'CDW',
 'CE',
 'COR',
 'CNC',
 'CNP',
 'CDAY',
 'CF',
 'CHRW',
 'CRL',
 'SCHW',
 'CHTR',
 'CVX',
 'CMG',
 'CB',
 'CHD',
 'CI',
 'CINF',
 'CTAS',
 'CSCO',
 'C',
 'CFG',
 'CLX',
 'CME',
 'CMS',
 'KO',
 'CTSH',
 'CL',
 'CMCSA